<a id="modeling"></a>
# Modeling and representation

**Representation learning** has deep connections with visualization - namely, dimensionality reduction allows us to project high dimensional data onto 2D or 3D planes so that we can visually inspect them.

But the main usefulness of representation (inspired) methods lies in their "combination" with classification and regression methods as "prior" "embedding" steps. This approach is mainly used in so called _kernel methods_.

## Kernel methods

### The advantages of "exploding the feature space"

Dimension reduction has many advantages, but other types of feature space transformations can also be useful in certain situations. For instance, as the case of polynomial regression demonstrated, "exploding the feature space" that is, transforming the data into a higher dimensional feature space can also serve us when we would like to use a model whose parameters are **linear in its input feature space** (e.g., linear regression or classification methods with linear decision boundaries) **to find nonlinear patterns** (e.g. nonlinear decision boundaries). The trick is, of course, to apply a nonlinear transformation to the data and use the method to find parameters (e.g. decision boundaries) that are linear in the new feature space but nonlinear in the original one:

<a href="https://journals.plos.org/ploscompbiol/article/figure/image?download&size=large&id=info:doi/10.1371/journal.pcbi.1000173.g006"><img  src="https://drive.google.com/uc?export=view&id=1a_gFPOQCq7Uk78jD6Ibqxe1Bm5P3MfCA"></a>



In its most general form, transforming the feature space is to use a new set of feature vectors $$\{\phi(\mathbf x_1),\dots,\phi(\mathbf x_N)\}$$ for training instead of the original $\{\mathbf x_1,\dots,\mathbf x_N\}$ data where $\phi$ is any function mapping vectors to vectors. In the case of a one-variable polynomial regression $\phi$ is simply the mapping

$$
\phi(\langle x\rangle) = \langle x, x^2,\dots,x^m \rangle.
$$


<a href="https://pbs.twimg.com/media/DJJKZR2XgAAliTi.jpg"><img src="https://drive.google.com/uc?export=view&id=1V4r6zWg8Q41W0MtsP8ZfgjijCQrcmJgU" width=400 heigth=400></a>

### But what about the curse of dimensionality?
When $\phi$ increases the number of dimensions/features we may be faced with the by now familiar problems of high dimensionality:

1. ML on high dimensional data is computationally expensive (both for memory and time, and frequently prohibitively so).
2. In the new high dimensional feature space our data becomes sparse and this can lead to overfitting.

We will return the second problem later, but, somewhat surprisingly, the first concern in many cases can be addressed by a clever move: Instead of _explicitly_ expanding the feature space, we can **keep working with the original data and modify our algorithms to work in a way which _implicitly_ assumes an expanded feature space**, but does not have the computational cost of explicit expansion. Methods using this solution are called "**kernel methods**" and the technique is frequently called the "**kernel trick**". 

### The kernel trick

The main observation on which the technique relies is that many ML algorithms (e.g., linear regression, SVM) can be rewritten in a form which does not refer to vectors in the input feature space explicitly but only implicitly via their _dot product_. For algorithms rewritten in this, so called _kernelized_ form it is enough to replace all references to the dot products of vectors in the input space, e.g., $\mathbf x \cdot \mathbf y$ with $\phi(\mathbf x) \cdot \phi(\mathbf y)$ to get a version which implicitly works in the transformed feature space, but does not require explicit feature mapping.

To demonstrate how accessing a high dimensional feature space only implicitly, in the form of dot products, can be useful, consider the feature map

$$
\phi(\langle x_1,x_2 \rangle) = \langle 1, x_1^2, x_2^2, \sqrt 2 x_1, \sqrt 2 x_2, \sqrt 2 x_1 x_2 \rangle.
$$

Because of the presence of the combined feature $\sqrt 2 x_1 x_2$ and quadratic features $x_1^2$ and $x_2^2$, this seems like a useful nonlinear feature expansion, albeit a bit complex. But much of the complexity vanishes if we have to calculate only the dot product of two transformed vectors, since

$$
\phi(\mathbf x) \cdot \phi(\mathbf y) = \langle 1, x_1^2, x_2^2, \sqrt 2 x_1, \sqrt 2 x_2, \sqrt 2 x_1 x_2 \rangle \cdot 
\langle 1, y_1^2, y_2^2, \sqrt 2 y_1, \sqrt 2 y_2, \sqrt 2 y_1 y_2 \rangle =
$$

$$
1 + x_1^2 y_1^2 + x_2^2 y_2^2 + 2x_1 y_1  + 2 x_2 y_2  + 2 x_1 y_1 x_2 y_2 = (1 + x_1 y_1 + x_2 y_2)^2 = (1 + \mathbf x \cdot \mathbf y)^2
$$
that is, compared to calculating $\mathbf x \cdot \mathbf y$ **we have only a simple scalar addition and multiplication as added difficulty**, but in exchange we are **_implicitly_ operating over a 6-dimensional quadratic feature space**.


### Kernel functions
The two-variable $K(\mathbf x,\mathbf y) = \phi(\mathbf x) \cdot \phi(\mathbf x)$ 
functions with which dot products are replaced when the kernel trick is applied are collectively known as kernel functions or kernels for short: any function is a kernel if it can be written in this form with some $\phi$ feature map. 

In addition to the above straightforward characterization, there are other, way more technical conditions of "kernelhood" that do not require explicitly this form but from which the possibility  of rewriting the function into this form follows (although the underlying feature mapping is not guaranteed to be especially tidy or even finite dimensional!!!), most notably that the function should be [positive semi-definite](https://en.wikipedia.org/wiki/Positive-definite_function) or, equivalently, should satisfy [Mercer's condition](https://en.wikipedia.org/wiki/Mercer%27s_theorem#Mercer%27s_condition). Unfortunately, the details are beyond the scope of this course.

The advantage of these more abstract characterizations is that they can be used to prove 

- that certain functions are kernels for which this is far from obvious (e.g. Gaussian kernels, see below).
- some general properties of kernels that are very useful for constructing complex kernels from simpler ones: The most important fact here is that
> any positive linear combination of kernels is also a kernel.

**Warning:** Sometimes the literature uses "kernel" in a much more general sense to denote any two-variable real valued function which is used as a similarity measure between its arguments.

### Kernelization and instance based methods
An important consequence of kernelization for many algorithms is that they become instance-based, i.e., in addition to learned parameters, they also need to store some individual training examples in memory in order to be able to predict.

The reason is that in many cases the non-kernelized versions of these algorithms learn one or more vectors in the feature space as parameters (e.g. a normal vector of a separating hyperplane), and these **parameters are no longer explicitly representable after kernelization**. The typical solution is to prove a so called **representer theorem** stating that such learned parameters can be written as linear combination of vectors in the image of training data according to the kernel's implicit feature mapping, that is, for those $p$ parameters

$$
p = \sum_{i=1}^N \alpha_i\phi(\mathbf x_i)
$$

for some $\alpha_i$ parameters and the training data. 

For many kernelized models and datasets **most of the $\alpha_i$-s will be $0$**, i.e., $p$ will be a linear combination of the value of $\phi$ for a few of the data points. Since **the learned parameters are used only in dot products with other $\phi$ values** (e.g. when calculating on which side of a hyperplane a point's image is located in a binary classifier), the explicit computation of $\phi$ values can be **avoided by the kernel trick**:

$$
p \cdot \phi(\mathbf z) = \left(\sum_{i=1}^N \alpha_i\phi(\mathbf x_i)\right) \cdot \phi(\mathbf z) = \sum_{i=1}^N \alpha_i\phi(\mathbf x_i) \cdot \phi(\mathbf z) = \sum_{i=1}^N \alpha_i K(\mathbf x_i, \mathbf z)
$$
but this still requires the storage of the non-zero $\alpha_i$ and $\mathbf x_i$ components of $p$'s representation, i.e. **the storage and use of some of the training data**.

### ML algorithms/methods with kernelized versions
A huge number of popular ML algorithms/methods have kernelized versions. Some of the most important ones are
* SVM (!!! this is by far the most important one)
* Ridge regression
* Logistic regression
* Perceptron (which we will meet later)
* PCA
* Linear discriminant analysis
* K-means

### Popular kernels

Some of the widely used kernels are

* **Linear kernel**: $K(\mathbf x, \mathbf y)=\mathbf x \cdot \mathbf y$: This is the kernel without any feature mapping (or with the identity feature mapping), which is used with kernelized algorithms when no feature mapping is needed.
* **Polynomial kernels**: kernels of the form $K(\mathbf x, \mathbf y) = (1+ \mathbf x \cdot \mathbf y)^n$ where $n$ is an integer -- these kernels correspond to polynomial feature mappings (we have seen an instance as an example above).
* **Gaussian or RBF (Radial Basis Function) kernels**: kernels of the form 
$$K(\mathbf x, \mathbf y) = \exp(-\frac{\|\mathbf x-\mathbf y\|^2}{2\sigma^2}).$$
Can be seen as inducing a nonlinear, Gaussian weighted distance metric on the original feature space. On the other hand, the "implicit feature mapping" behind them is infinite dimensional as can be shown by using the Taylor series expansion of the exponential function. (See, e.g., [these slides](https://www.csie.ntu.edu.tw/~cjlin/talks/kuleuven_svm.pdf) for details.)
* **String kernels**: These kernels operate on strings and measure their similarity in various ways, e.g., they can measure the number of substrings that occur in both of them (strings are from alphabet $A$):
$$K(\mathbf x, \mathbf y) = \sum_{s\in A^*}w_s c_s(\mathbf x)c_s(\mathbf y)$$
where $c_s(\mathbf x)$ is the number of occurrences of $s$ in $\mathbf x$ as a substring, and $w_s$ is a weight belonging to $s$. Similarly to the Gaussian kernel, the underlying feature space has an inifinite number of dimensions but here -- in contrast to the Gaussian -- the used feature mapping is fairly obvious.

<a href="https://qph.fs.quoracdn.net/main-qimg-c7f5c6f1fc6d4be7daaaf82d975e226e"><img src="https://drive.google.com/uc?export=view&id=18m62kM_rPK7PHTz29W8D_XVAEUWwcVxK" width=500 heigth=500></a>


### Kernel methods and overfitting
Fortunately, despite the implicit use of high dimensional feature spaces, the usual means of avoiding overfitting, i.e.,  regularization and careful, cross-validation based hyperparameter tuning work well with kernel methods. Moreover, in the case of kernel SVM, which is the most frequently used kernel method, experience and some theoretical results from learning theory show that the (properly set) max margin objective already acts as a very strong regularizer (capacity limiter), see e.g. [this presentation](https://web.archive.org/web/20170119042122/http://www.cs.utah.edu/~piyush/teaching/27-9-print.pdf) for some details.

<a id="repragain"></a>
# Understanding representation learning

- **Memorization:**

Like KNN, it is pure "remembering", basically a database learner, or a "lookup table", where we just query the similar items. The database is itself its own model, no compression.

If we think about it, even a linear model captures "the essence" of a relationship in the data, every point contributes to the "memory" - good illustration: problems with outliers and robust regression methods - remembering the "false" signal, even with high "leverage".

We can conclude: every model memorizes. The question is: For any kind of more abstract model, we would like it to memorize "the gist of the relationship", that is to "generalize". (See problem of overfitting.) To avoid overfitting, we would like the model to be the most "concise" description of the relationship (Occam factor).

- **Compression / hashing (capturing most salient features)**

Concisely describing something can be understood as a form of lossful compression. We can compress something if we can substitute its description to a more compact one (see the whole literature on information theory and compression), which is only possible if we capture something deeply descriptive about the phenomena. To put it another way, if we capture the "salient" features of the data, we can describe it concisely, thus our "understanding" (model) of it is good. This is the motivation behind the [Hutter prize](https://en.wikipedia.org/wiki/Hutter_Prize) which is a prize for compressing 100Mb of Wikipedia texts - and it *is* an AI prize!

[Hashing](https://en.wikipedia.org/wiki/Hash_function) is in connection with compression ("mapping data of arbitrary size to data of a fixed size"), meaning the systematic mapping of one representation to another, which can be understood as a function.

- **Function application on space**

Learning or engineering of features from raw data is the creation of (successive) transformations (in the form of functions) that are applied to the data to bring its salient features into the foreground, so as to enable its compression, representation in a lower dimensional space. 

- **"Embed and cut"**

Since the lower dimensional space of features captures the essence of phenomena, it is suitable as an input for a (linear) classification. The "signal" is the distance / separability of classes, the "noise" is the scatteredness of classes. (Connection with [Fisher information](https://en.wikipedia.org/wiki/Fisher_information#Matrix_form).)

**Conclusion: you need only a simple classifier, but over a very good (learned representation).**

**_Representation is everything_.**

# Summary of "classical" methods

Scikit has a nice - but naturally very oversimplified - "decision tree" about the estimators it incorporates. Many times it is worth checking. 

<a href="https://scikit-learn.org/stable/_static/ml_map.png"><img src="https://drive.google.com/uc?export=view&id=1gMslwxuLSZp_PfooZLjYGFyZKxhlktPg" width=85%></a>

[Source](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)

But for more complex, high dimensional problems we will have to go further in the direction of **deep learning**.